In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!unzip ../input/facial-keypoints-detection/test.zip -d /kaggle/working/
!unzip ../input/facial-keypoints-detection/training.zip -d /kaggle/working/

In [ ]:
test = pd.read_csv("./test.csv")
train = pd.read_csv("./training.csv")

In [ ]:
train.columns

In [ ]:
train =train.dropna()

In [ ]:
df_without_images = train.drop("Image",1)
images = train["Image"]


In [ ]:
df_without_images.head()

In [ ]:
img_data = []
for img in images:
    img = [int(i) for i in img.split(" ")]
    img = np.array(img)
    img = img.reshape(96,96,1)
    img_data.append(img)
img_data = np.array(img_data)

In [ ]:
keypoint_features = []
for idx, features in df_without_images.iterrows():
    keypoint_features.append(features)
keypoint_features = np.array(keypoint_features, dtype=float)

In [ ]:
img_data.shape

In [ ]:
rand_indexes = np.random.randint(0,img_data.shape[0],9)
print(rand_indexes)
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(3*4, 3*4))
for r in range(3):
    for c in range(3):
        idx = r * 3 + c
        axes[r, c].imshow(img_data[rand_indexes[r+c]])
        axes[r, c].scatter(keypoint_features[rand_indexes[r+c]][0::2], keypoint_features[rand_indexes[r+c]][1::2], s=20,c="white")

In [ ]:
norm_images = img_data / 255.0
norm_keypoints = keypoint_features / 96.0

In [ ]:
import tensorflow as tf
tf.random.set_seed(101)
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense,Conv2D,Flatten,Dropout,Activation,BatchNormalization
from tensorflow.keras.models import Sequential,Model,load_model
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler,ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
def conv2d_model():
    model = Sequential()
    
    model.add(Conv2D(3, (1,1), padding='same', input_shape=(96,96,1)))
    pretrained_model = MobileNetV2(input_shape=(96,96,3), include_top=False, weights='imagenet')
    pretrained_model.trainable = True
    model.add(pretrained_model)
    
    model.add(Flatten())
    #flatten = GlobalAveragePooling2D()(flatten)
    model.add(Dense(1024, activation="relu"))
    model.add(Dense(512, activation="relu"))
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation="relu"))
    model.add(Dense(64, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation="relu"))

    model.add(Dense(30))
    model.compile(loss='mse', optimizer=Adam(learning_rate =0.003),metrics=['accuracy',"mse"])

    return model

In [ ]:
model = conv2d_model()

In [ ]:
model.summary()

In [ ]:
EarlyStopper = tf.keras.callbacks.EarlyStopping(monitor='mse', patience=10)
checkpoint_path_quality = f"/kaggle/working/face_kp.h5"

checkpoint = ModelCheckpoint(checkpoint_path_quality, 
                             monitor='mse', 
                             verbose=1,
                             save_best_only=True, 
                             mode='min')
learning_rate_reduction = ReduceLROnPlateau(monitor='mse',
                                            patience=2,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.000001)

In [ ]:
history = model.fit(norm_images,norm_keypoints, batch_size = 16,
	epochs=20,callbacks = [checkpoint,learning_rate_reduction,EarlyStopper])

# Test

In [ ]:
test_images = test["Image"]

In [ ]:
test_img_data = []
for img in test_images:
    img = [int(i) for i in img.split(" ")]
    img = np.array(img)
    img = img.reshape(96,96,1)
    test_img_data.append(img)
test_img_data = np.array(test_img_data)
test_img_data = test_img_data/ 255.0

In [ ]:
loaded_model = load_model("/kaggle/working/face_kp.h5")

In [ ]:
test_predict = loaded_model.predict(test_img_data)

In [ ]:
test_predict = test_predict * 96

In [ ]:
rand_indexes = np.random.randint(0,test_img_data.shape[0],9)
print(rand_indexes)
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(3*4, 3*4))
for r in range(3):
    for c in range(3):
        idx = r * 3 + c
        axes[r, c].imshow(test_img_data[rand_indexes[r+c]])
        axes[r, c].scatter(test_predict[rand_indexes[r+c]][0::2], test_predict[rand_indexes[r+c]][1::2], s=20,c="white")

In [ ]:
idlookup_file = pd.read_csv('../input/facial-keypoints-detection/IdLookupTable.csv')
feature_names = list(idlookup_file['FeatureName'])
image_ids = list(idlookup_file['ImageId']-1)
row_ids = list(idlookup_file['RowId'])

feature_list = []
for feature in feature_names:
    feature_list.append(feature_names.index(feature))
    
predictions = []
for x,y in zip(image_ids, feature_list):
    predictions.append(test_predict[x][y])
    
row_ids = pd.Series(row_ids, name = 'RowId')
locations = pd.Series(predictions, name = 'Location')
locations = locations.clip(0.0,96.0)
submission_result = pd.concat([row_ids,locations],axis = 1)
submission_result.to_csv('submission.csv',index = False)